In [1]:
import urllib.request
from bs4 import BeautifulSoup as soup
import requests
import re
import pandas as pd
import dataframe_image as dfi


link = 'https://rally-base.com/2023/vodafone-rally-de-portugal-2023/?ssId='
startat, no_ss=7519, int(19) # starting number of url, count of Special Stages
canceled = []

stages = [number for number in range(no_ss)]


rally_23 = []

In [2]:
if canceled:
    for j in canceled: stages.remove(j-1)
#print(stages)


In [3]:
for ss in stages:
    val= startat + ss
    ss_a = str(val)
    my_url11 = link + ss_a
    #print(ss, val, ss_a, "\n", my_url11)
    
    req = urllib.request.Request(my_url11, headers={'User-Agent': 'Mozilla/5.0'})
    uClient11 = urllib.request.urlopen(req)
    page_html11 = uClient11.read()
    uClient11.close()
    data = pd.read_html(page_html11)[1]
   
    data.columns = data.iloc[0]
    data = data[1:]
    data['ss']=ss+1
    #print(data.columns)
    
    equal = '-' in data['Pos.'].unique()
    if equal:
        data['Pos.'] = data['Pos.'].replace('-', method='ffill')
    
    #data.to_csv('05_rally23_SS'+str(ss+1)+'.csv', index=False)

    rally_23.append(data)

In [4]:
rally2023_stages = pd.concat(rally_23, axis=0)
#rally2023_stages['Pos.'] = rally2023_stages['Pos.'].astype(str).astype(int)
rally2023_stages['No.'] = rally2023_stages['No.'].astype(str).astype(int)
#rally2023_stages.to_csv('03_rally2023.csv', index=False)
rally2023_stages = rally2023_stages.fillna("-")
rally2023_temp=rally2023_stages.drop(['Group','SS time', 'Aver. speed sec/km', 'Diff.Leader Diff.Prev.'], axis=1)
rally2023_view = rally2023_temp.set_index(['No.','Driver / Co-driver Vehicle','ss'], drop=True).unstack('ss')

rally2023_temp2 = rally2023_temp[(rally2023_temp['No.']<30)]
rally2023_temp3 = rally2023_temp[(rally2023_temp['No.']==69)]
#rally2023_temp2.append(rally2023_temp3)
#rally2023_temp4 = rally2023_temp2.append(rally2023_temp3)
rally2023_temp4 = pd.concat([rally2023_temp2, rally2023_temp3],axis=0)
rally2023_view2 = rally2023_temp4.set_index(['No.','Driver / Co-driver Vehicle','ss'], drop=True).unstack('ss')
rally2023_view2 = rally2023_view2.fillna("-")
rally2023_view2

0                                                      Pos.                  \
ss                                                       1   2   3   4   5    
No. Driver / Co-driver Vehicle                                                
4   Lappi  Esapekka  /  Ferm  Janne  Hyundai i20 N ...    6   7   7   1   5   
6   Sordo  Dani  /  Carrera  C.  Hyundai i20 N Rall...    2   2   4   2   3   
7   Loubet  P.  /  Gilsoul  Nicolas  Ford Puma Rall...    1   5   6   3   4   
8   Tänak  Ott  /  Järveoja  M.  Ford Puma Rally1 H...    3   1   2  32   6   
11  Neuville  T.  /  Wydaeghe  M.  Hyundai i20 N Ra...    7   6   5   5   2   
18  Katsuta  T.  /  Johnston  Aaron  Toyota GR Yari...    5   3  62  58  56   
20  Greensmith  Gus  /  Andersson  Jonas  Škoda Fab...   15  19  13   9  11   
21  Solberg  Oliver  /  Edmondson  E.  Škoda Fabia ...   13  11  10   7   8   
22  Gryazin  Nikolay  /  Aleksandrov  K.  Škoda Fab...   14  14  14  59  57   
23  Suninen  Teemu  /  Markkula  Mikko  Hyundai i20...    9  12  11   8   9   
24  Mikkelsen  A.  /  Eriksen  T.  Škoda Fabia RS R...   16  10   8  50  10   
25  Fourmaux  Adrien  /  Coria  Alexandre  Ford Fie...   10   9   9  11  39   
26  Rossel  Yohan  /  Dunand  Arnaud  Citroën C3 Ra...   11  13  12  10  12   
27  Zaldivar  F.  /  Der Ohannesian  M.  Hyundai i2...   21  21  22  59  57   
28  Munster  G.  /  Louka  Louis  Ford Fiesta Rally...   64  63  63  59  57   
29  Virves  Robert  /  Magalhães  Hugo  Ford Fiesta...   44  24  24  18  15   
69  Rovanperä  K.  /  Halttunen  Jonne  Toyota GR Y...    4   4   1   4   1   

0                                                                           \
ss                                                      6   7   8   9   10   
No. Driver / Co-driver Vehicle                                               
4   Lappi  Esapekka  /  Ferm  Janne  Hyundai i20 N ...   5   1   5   2   3   
6   Sordo  Dani  /  Carrera  C.  Hyundai i20 N Rall...   2   6   1   3   6   
7   Loubet  P.  /  Gilsoul  Nicolas  Ford Puma Rall...   3   4   4   6  64   
8   Tänak  Ott  /  Järveoja  M.  Ford Puma Rally1 H...   6   3   2   5   2   
11  Neuville  T.  /  Wydaeghe  M.  Hyundai i20 N Ra...   4   5   2   4   4   
18  Katsuta  T.  /  Johnston  Aaron  Toyota GR Yari...  50  47  48   7   5   
20  Greensmith  Gus  /  Andersson  Jonas  Škoda Fab...  12   9   7   9  10   
21  Solberg  Oliver  /  Edmondson  E.  Škoda Fabia ...  10   8  16  14  13   
22  Gryazin  Nikolay  /  Aleksandrov  K.  Škoda Fab...  51  49  50  13  14   
23  Suninen  Teemu  /  Markkula  Mikko  Hyundai i20...  38  13   8  12   8   
24  Mikkelsen  A.  /  Eriksen  T.  Škoda Fabia RS R...   8   7   9  11   9   
25  Fourmaux  Adrien  /  Coria  Alexandre  Ford Fie...  48  10   9   8   7   
26  Rossel  Yohan  /  Dunand  Arnaud  Citroën C3 Ra...   9  12  11  19  16   
27  Zaldivar  F.  /  Der Ohannesian  M.  Hyundai i2...  51  49  50  18  20   
28  Munster  G.  /  Louka  Louis  Ford Fiesta Rally...  51  49  50  30  46   
29  Virves  Robert  /  Magalhães  Hugo  Ford Fiesta...  42  11  12  22  47   
69  Rovanperä  K.  /  Halttunen  Jonne  Toyota GR Y...   1   2   6   1   1   

0                                                                           \
ss                                                      11  12  13  14  15   
No. Driver / Co-driver Vehicle                                               
4   Lappi  Esapekka  /  Ferm  Janne  Hyundai i20 N ...   5   3   4   2   4   
6   Sordo  Dani  /  Carrera  C.  Hyundai i20 N Rall...   3   2   1   5   1   
7   Loubet  P.  /  Gilsoul  Nicolas  Ford Puma Rall...  63  60  53  54  52   
8   Tänak  Ott  /  Järveoja  M.  Ford Puma Rally1 H...   4   5   6   4   5   
11  Neuville  T.  /  Wydaeghe  M.  Hyundai i20 N Ra...   2   4   3   3   3   
18  Katsuta  T.  /  Johnston  Aaron  Toyota GR Yari...  25   6   5   7   2   
20  Greensmith  Gus  /  Andersson  Jonas  Škoda Fab...   7   9   9   8  15   
21  Solberg  Oliver  /  Edmondson  E.  Škoda Fabia ...  12  10  12   6   8   


In [5]:
pd.set_option("display.max_columns", None)
rally2023_view2
dfi.export(rally2023_view2, 'Portugal_Stages_Classification.jpg')